<div class="alert alert-block alert-danger">
<h2>Table of Contents</h2>
</div>

<a href='#1.0'>Import libraries</a><br>
<a href='#2.0'>Connect to the mysql database</a><br>
<a href='#3.0'>Load data</a><br>
<a href='#4.0'>Dataframes to SQL</a><br>
<a href='#5.0'>Main Dataframe replaced with index values to SQL</a><br>

<a id='1.0'></a>
<div class="alert alert-block alert-danger">
<h2>Import Libraries</h2>
</div>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # ignore information messages

import math
import numpy as np 
import pandas as pd
# display all of the columns
pd.set_option('display.max_columns', None)

<a id='2.0'></a>
<div class="alert alert-block alert-danger">
<h2>Connect to the mysql database</h2>
</div>

<div class="alert alert-block alert-warning">
        Read the cvc file with username and password
</div>

In [2]:
mysql_user_pass = pd.read_csv('../../../mysql_user_pass.csv', index_col=0)
username = mysql_user_pass['mysql_user'][0]
password = mysql_user_pass['mysql_pass'][0]

<div class="alert alert-block alert-warning">
        Create an engine to connect to the mysql database
</div>

In [3]:
from sqlalchemy import create_engine

engine = create_engine(f'mysql+pymysql://{username}:\
{password}@localhost:3306/Steel_Industry_Energy_Consumption')

<a id='3.0'></a>
<div class="alert alert-block alert-danger">
<h2>Load data</h2>
</div>

At EMP we deal daily with a great amount of data from different sources (from web, shop, inventory, purchasing, logistics, finance, etc.), produced by the customer journey.
In this challenge, we will focus on the sales data, produced by customers visiting and purchasing items from our web shop (emp.de in Germany).

In [4]:
df = pd.read_csv('../../../all_dfs/Steel_Industry_Energy_Consumption/Steel_industry_data.csv')
df.head()

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


<div class="alert alert-block alert-warning">
        Parse the date feature
</div>

In [5]:
df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%Y %H:%M')

<div class="alert alert-block alert-success">
    <h4>
        Check categorical columns
    </h4>
</div>

In [6]:
categorical_feature_cols = [i for i in df.columns if df[i].dtype == "object"]
print(categorical_feature_cols)

['WeekStatus', 'Day_of_week', 'Load_Type']


<a id='4.0'></a>
<div class="alert alert-block alert-danger">
<h2>Dataframes to SQL</h2>
</div>

In [7]:
from sklearn.preprocessing import LabelEncoder

def uniquestosql(train, col, test=pd.DataFrame()):
    
    le = LabelEncoder()
    
    # encode items but not nans
    train_wo_null = train[col][train[col].notnull()]
    train[col] = pd.Series(le.fit_transform(train_wo_null),index=train_wo_null.index)
    
    # create a dataframe with unique values to send to sql
    train_unique = pd.DataFrame(data=le.classes_, index=le.transform(le.classes_), columns=[col])

    # send the dataframe to sql
    # if_exists{‘fail’, ‘replace’, ‘append’}, default ‘fail’
    train_unique.to_sql(col, con=engine, if_exists='replace')
    
    if len(test)>0:
        test_wo_null = test[col][test[col].notnull()]
        test[col] = pd.Series(le.fit_transform(test_wo_null),index=test_wo_null.index)
        
        return train, test
    return train

In [8]:
for col in categorical_feature_cols:
    df = uniquestosql(df, col)

In [9]:
df.head(3)

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,2018-01-01 00:15:00,3.17,2.95,0.0,0.0,73.21,100.0,900,0,1,0
1,2018-01-01 00:30:00,4.00,4.46,0.0,0.0,66.77,100.0,1800,0,1,0
2,2018-01-01 00:45:00,3.24,3.28,0.0,0.0,70.28,100.0,2700,0,1,0


<a id='5.0'></a>
<div class="alert alert-block alert-danger">
<h2>Main Dataframe replaced with index values to SQL</h2>
</div>

In [10]:
df.to_sql('Main_table', con=engine, if_exists='replace')